In [1]:
using Pkg
Pkg.activate("..")

  Activating environment at `~/avg/WoR/world_on_rails.jl/Project.toml`


In [2]:
using Revise
using Test
using Knet
using Statistics:mean
using NPZ

In [3]:
push!(LOAD_PATH, joinpath(dirname(pwd()), "src", "data"))
push!(LOAD_PATH, joinpath(dirname(pwd()), "src", "models"))

5-element Vector{String}:
 "@"
 "@v#.#"
 "@stdlib"
 "/home/enes/avg/WoR/world_on_rails.jl/src/data"
 "/home/enes/avg/WoR/world_on_rails.jl/src/models"

In [4]:
cd("..")

## DataLoader Main Model

In [5]:
import main_dataset: LabeledMainDataset
main_dataset = LabeledMainDataset("/home/enes/avg/WoR/world_on_rails.jl/data/main_mini",
     "/home/enes/avg/WoR/world_on_rails.jl/configs/config.yaml");

/home/enes/avg/WoR/world_on_rails.jl/data/main_mini: 172 frames (x3)


In [6]:
(wide_rgbs, wide_sems, narr_rgbs, narr_sems, act_vals, spds, cmds) = main_dataset[1];

In [7]:
wide_rgbs = reshape(wide_rgbs, (1,size(wide_rgbs)...));
narr_rgbs = reshape(narr_rgbs, (1,size(narr_rgbs)...));
act_vals = reshape(act_vals, (1,size(act_vals)...));

In [8]:
act_vals[1,:,:,:]

28×4×6 Array{Any, 3}:
[:, :, 1] =
 0.957224  0.932526  0.428275  0.237606
 0.971438  1.15929   0.664114  0.415488
 0.980962  1.24901   0.923563  0.574927
 0.987574  1.30826   1.0457    0.734809
 0.989072  1.32588   1.08133   0.771495
 0.957188  1.14214   0.805325  0.562069
 0.921557  0.949503  0.581062  0.456128
 0.878715  0.733887  0.462459  0.334281
 0.824732  0.577492  0.326122  0.184719
 0.961238  0.932227  0.427769  0.237606
 0.975497  1.15964   0.661975  0.415488
 0.98505   1.24974   0.919751  0.574927
 0.991683  1.30932   1.04132   0.734809
 ⋮                             
 0.882419  0.732998  0.462012  0.334281
 0.828241  0.57708   0.32608   0.184719
 0.965253  0.931929  0.427263  0.237606
 0.979556  1.15998   0.659836  0.415488
 0.989139  1.25047   0.915939  0.574927
 0.995792  1.31037   1.03694   0.734809
 0.997293  1.32826   1.07209   0.771495
 0.965171  1.14268   0.799652  0.562069
 0.929276  0.948628  0.578681  0.456128
 0.886122  0.732109  0.461565  0.334281
 0.83175   0.5

In [9]:
@test size(wide_rgbs) == (1,193,480,3)
@test size(narr_rgbs) == (1,160,384,3)
@test size(act_vals) == (1, 28, 4, 6)

Test Passed

In [10]:
wide_rgbs = KnetArray{Float32}(permutedims(Float32.(wide_rgbs), (1, 4, 2, 3)))
narr_rgbs = KnetArray{Float32}(permutedims(Float32.(narr_rgbs), (1, 4, 2, 3)))
wide_sems = KnetArray{Int64}(Int64.(wide_sems))
narr_sems = KnetArray{Int64}(Int64.(narr_sems));

## softmax test

In [11]:
act_vals = KnetArray{Float32}(permutedims(act_vals, (1, 3, 2, 4)));

In [12]:
act_probs = Knet.softmax(act_vals./0.01, dims = 3);

In [13]:
@test act_probs[1,1,1,1] ≈ 0.0031278

Test Passed

## kl_div tests

### act_output mock reading

In [14]:
act_outputs = npzread(joinpath("test", "mock_output", "act_outputs.npy"));

In [15]:
act_outputs = permutedims(act_outputs, (1,3,4,2));

### logsoftmax test

In [23]:
prob_act_outputs = logsoftmax(act_outputs, dims=3);

In [24]:
@test prob_act_outputs[1,1,1,1] == -3.4147465f0

Test Passed

In [19]:
using metrics: kl_div

KnetArray{Float32, 4}

In [70]:
kl = kl_div(KnetArray(prob_act_outputs), KnetArray(act_probs));

In [73]:
mean(mean(kl, dims=2), dims=3)

1×1×1×6 KnetArray{Float32, 4}:
[:, :, 1, 1] =
 0.05640474

[:, :, 1, 2] =
 0.05589357

[:, :, 1, 3] =
 0.055642966

[:, :, 1, 4] =
 0.05554856

[:, :, 1, 5] =
 0.04199929

[:, :, 1, 6] =
 0.04567174

In [63]:
x = 

1.0000000000000022e-40